### Proyecto Individual No 1 Data Engineering

In [10]:
import pandas as pd
import numpy as np

#### Extraccón de datos desde los archivos csv

In [7]:
amazon = pd.read_csv('./Datasets/amazon_prime_titles-score.csv')
disney = pd.read_csv('./Datasets/disney_plus_titles-score.csv')
hulu = pd.read_csv('./Datasets/hulu_titles-score.csv')
netflix = pd.read_csv('./Datasets/netflix_titles-score.csv')

#### Transformaciones

* 1. Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123

In [11]:
disney.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!,100


In [8]:
amazon.insert(loc=0, column= 'id', value = 'a' + amazon.show_id)
disney.insert(loc=0, column= 'id', value = 'd' + disney.show_id)
hulu.insert(loc=0, column= 'id', value = 'h' + hulu.show_id)
netflix.insert(loc=0, column= 'id', value = 'n' + netflix.show_id)

In [13]:
netflix.head(1)

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,ns1,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",29


* 2. Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

In [9]:
amazon.rating.fillna('G',inplace=True)
disney.rating.fillna('G',inplace=True)
hulu.rating.fillna('G',inplace=True)
netflix.rating.fillna('G',inplace=True)

In [15]:
amazon.rating.isnull().sum()

0

* 3. De haber fechas, deberán tener el formato AAAA-mm-dd

In [16]:
amazon.date_added[0]

'March 30, 2021'

In [11]:
amazon.date_added.isna().sum()

9513

In [12]:
# Imputación de valores nulos por '0'
amazon.date_added.fillna('0',inplace=True)
disney.date_added.fillna('0',inplace=True)
hulu.date_added.fillna('0',inplace=True)
netflix.date_added.fillna('0',inplace=True)

In [13]:
amazon.date_added.isna().sum()

0

In [18]:
# Función para convertir el formato de la fecha a %Y-%m-%d
from datetime import datetime
def convert_date(column):
    fechas = []
    for i in column:
        if str(i)=='0':
            fechas.append(i)
        else:
            objdate = datetime.strptime(str(i.strip()), "%B %d, %Y")
            fechas.append(objdate.strftime("%Y-%m-%d"))
    return fechas   

In [19]:
#Se crea la columna date
amazon['date']=pd.DataFrame(convert_date(amazon.date_added))
disney['date']=pd.DataFrame(convert_date(disney.date_added))
hulu['date']=pd.DataFrame(convert_date(hulu.date_added))
netflix['date']=pd.DataFrame(convert_date(netflix.date_added))

In [20]:
netflix.head(1)

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score,date
0,ns1,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",29,2021-09-25


* 4. Los campos de texto deberán estar en minúsculas, sin excepciones

In [21]:
amazon = amazon.applymap(lambda x : x.lower() if type(x) == str else x)
disney = disney.applymap(lambda x : x.lower() if type(x) == str else x)
hulu = hulu.applymap(lambda x : x.lower() if type(x) == str else x)
netflix = netflix.applymap(lambda x : x.lower() if type(x) == str else x)


In [22]:
amazon.head(1)

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score,date
0,as1,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,"march 30, 2021",2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,99,2021-03-30


* 5. El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [24]:
# Imputación de valores nulos de la columna duration
amazon.duration.fillna('0',inplace=True)
disney.duration.fillna('0',inplace=True)
hulu.duration.fillna('0',inplace=True)
netflix.duration.fillna('0',inplace=True)

In [25]:
#Función para extraer el tiempo de la cadena de caracteres
from collections import deque
import re
def duration_int(column):
    d_int = []
    for i in column:
        if i == '0':
            d_int.append(i)
        else:
            str1= re.split('[a-zA-Z]',str(i))
            d_int.append(str1[0])
    return d_int

In [26]:
# Se crea la columna duration_int
amazon['duration_int']=duration_int(amazon.duration)
disney['duration_int']=duration_int(disney.duration)
hulu['duration_int']=duration_int(hulu.duration)
netflix['duration_int']=duration_int(netflix.duration)


In [28]:
netflix.head(1)

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score,date,duration_int
0,ns1,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,"september 25, 2021",2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",29,2021-09-25,90


In [29]:
#Función para extraer el tipo de duraciónde la cadena de caracteres
from collections import deque
def duration_type(column):
    d_type = []
    for i in column:
        if i == '0':
            d_type.append(i)
        else:
            str1=  re.split('[0-9]',i)
            str1 = deque(str1)
            str1.rotate(1)
            d_type.append(str1[0])
    return d_type

In [30]:
# Se crea la columna duration_type
amazon['duration_type']=duration_type(amazon.duration)
disney['duration_type']=duration_type(disney.duration)
hulu['duration_type']=duration_type(hulu.duration)
netflix['duration_type']=duration_type(netflix.duration)

In [34]:
netflix.head(1)

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score,date,duration_int,duration_type
0,ns1,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,"september 25, 2021",2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",29,2021-03-30,90,min


In [ ]:
netflix.info()

#### Carga de datos en una base de datos MySQL

In [33]:
import pandas.io as sql
import sqlalchemy as db
import pymysql

In [34]:
# Credenciales de la base de datos creada en Free MySQL Hosting
user = 'sql10591684'
password = '2vv82khQDF'
host = 'sql10.freemysqlhosting.net'
port = 3306
database = 'sql10591684'
database_connection = db.create_engine(url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database))

In [ ]:
connection = database_connection.connect()
metadata = db.MetaData()

In [35]:
amazon.head(1)

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score,date,duration_int,duration_type
0,as1,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,"march 30, 2021",2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,99,2021-03-30,113,min


In [236]:
#Se seleccionas las columnas de interes para el almacenamiento en la base de datos
amazon=amazon[['id','title','type','country','release_year','rating','score','duration_int','duration_type']]

In [237]:
#Se eliminan espacios vacios y se convierte la columna duration_int a entero
amazon['duration_int']=amazon['duration_int'].str.strip()
amazon['duration_int']=amazon['duration_int'].astype(int)
amazon['duration_type']=amazon['duration_type'].str.strip()

In [238]:
#Se envía la tabla amazon a la base de datos
amazon.to_sql('amazon',connection)

9668

In [239]:
disney=disney[['id','title','type','country','release_year','rating','score','duration_int','duration_type']]

In [240]:
disney['duration_int']=disney['duration_int'].str.strip()
disney['duration_int']=disney['duration_int'].astype(int)
disney['duration_type']=disney['duration_type'].str.strip()

In [241]:
#Se envía la tabla disney a la base de datos
disney.to_sql('disney',connection)

1450

In [242]:
hulu=hulu[['id','title','type','country','release_year','rating','score','duration_int','duration_type']]
hulu['duration_int']=hulu['duration_int'].str.strip()
hulu['duration_int']=hulu['duration_int'].astype(int)
hulu['duration_type']=hulu['duration_type'].str.strip()

In [243]:
#Se envía la tabla hulu a la base de datos
hulu.to_sql('hulu',connection)

3073

In [244]:
netflix=netflix[['id','title','type','country','release_year','rating','score','duration_int','duration_type']]
netflix['duration_int']=netflix['duration_int'].str.strip()
netflix['duration_int']=netflix['duration_int'].astype(int)
netflix['duration_type']=netflix['duration_type'].str.strip()

In [245]:
#Se envía la tabla netflix a la base de datos
netflix.to_sql('netflix',connection)

8807

In [219]:
# DEFINE THE DATABASE CREDENTIALS
user = 'sql10591684'
password = '2vv82khQDF'
host = 'sql10.freemysqlhosting.net'
port = 3306
database = 'sql10591684'
  
# PYTHON FUNCTION TO CONNECT TO THE MYSQL DATABASE AND
# RETURN THE SQLACHEMY ENGINE OBJECT
def get_connection():
    return db.create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )
  
  
if __name__ == '__main__':
  
    try:
        
        # GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
        engine = get_connection()
        print(
            f"Connection to the {host} for user {user} created successfully.")
    except Exception as ex:
        print("Connection could not be made due to the following error: \n", ex)

Connection to the sql10.freemysqlhosting.net for user sql10591684 created successfully.
